In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

# Importing Data

#### Initial cleaning of dataset 'movies' was done on R, where another dataset (movies_metadata) with movie summaries was joined, missing values handled, and columns formatted

In [2]:
# Initial cleaning of dataset movies was done on R, where another dataset with movie summaries was joined, missing values handled,
# and columns formatted
md = pd. read_csv('C:/Users/Mrinal/Downloads/serendipity-sac2018/serendipity-sac2018/movies.csv')
#contains movie tagwords
tag = pd. read_csv('C:/Users/Mrinal/Downloads/serendipity-sac2018/serendipity-sac2018/tags.csv')

#contains movie tagwords with relevance
taggen = pd. read_csv('C:/Users/Mrinal/Downloads/serendipity-sac2018/serendipity-sac2018/tag_genome.csv')


#contains user ratings
train = pd. read_csv('C:/Users/Mrinal/Downloads/serendipity-sac2018/serendipity-sac2018/training.csv')
md.head()

,movieId,unformatted_title,title,titlewithyear,releaseDate,Year,director,starring,imdbId,tmdbId,genres,summary
0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Adventure,Animation,Children,Comedy,Fantasy","Led by Woody, Andy's toys live happily in his ..."
1,2,Jumanji,Jumanji,Jumanji (1995),12/15/1995,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,"Adventure,Children,Fantasy",When siblings Judy and Peter discover an encha...
2,3,Grumpier Old Men,Grumpier Old Men,Grumpier Old Men (1995),1/1/1995,1995.0,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,"Comedy,Romance",A family wedding reignites the ancient feud be...
3,4,Waiting to Exhale,Waiting to Exhale,Waiting to Exhale (1995),1/15/1996,1996.0,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,31357.0,"Comedy,Drama,Romance","Cheated on, mistreated and stepped on, the wom..."
4,5,Father of the Bride Part II,Father of the Bride Part II,Father of the Bride Part II (1995),12/8/1995,1995.0,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,11862.0,Comedy,Just when George Banks has recovered from his ...


In [3]:
train.head()

,userId,movieId,rating,timestamp
0,142882,91658,2.5,1515209647000
1,142882,4344,1.0,1515209646000
2,142882,45720,2.0,1515209643000
3,142882,4734,2.0,1515209641000
4,142882,91542,2.0,1515209637000


### calculate mean and count of ratings

In [4]:

a=pd.DataFrame(train.groupby('movieId')['rating'].mean())
b=pd.DataFrame(train.groupby('movieId')['rating'].count())


In [5]:
x=pd.merge(a,b,on='movieId')

In [6]:
x.columns=['vote_average', 'vote_count']

In [7]:
x.head()

,vote_average,vote_count
movieId,,
1,3.799308,20666
2,3.302115,6620
3,2.943738,1022
4,2.736641,131
5,2.887511,2218


In [8]:
md=pd.merge(md,x,on='movieId')

In [9]:
md.head()

,movieId,unformatted_title,title,titlewithyear,releaseDate,Year,director,starring,imdbId,tmdbId,genres,summary,vote_average,vote_count
0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Adventure,Animation,Children,Comedy,Fantasy","Led by Woody, Andy's toys live happily in his ...",3.799308,20666
1,2,Jumanji,Jumanji,Jumanji (1995),12/15/1995,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,"Adventure,Children,Fantasy",When siblings Judy and Peter discover an encha...,3.302115,6620
2,3,Grumpier Old Men,Grumpier Old Men,Grumpier Old Men (1995),1/1/1995,1995.0,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,"Comedy,Romance",A family wedding reignites the ancient feud be...,2.943738,1022
3,4,Waiting to Exhale,Waiting to Exhale,Waiting to Exhale (1995),1/15/1996,1996.0,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,31357.0,"Comedy,Drama,Romance","Cheated on, mistreated and stepped on, the wom...",2.736641,131
4,5,Father of the Bride Part II,Father of the Bride Part II,Father of the Bride Part II (1995),12/8/1995,1995.0,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,11862.0,Comedy,Just when George Banks has recovered from his ...,2.887511,2218


### using IMDB's weighter average formula

In [10]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
m = vote_counts.quantile(0.95)
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()


In [11]:
m

752.0

In [12]:
C

2.665018616858939

In [13]:
qdf = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'Year', 'vote_count', 'vote_average', 'genres']]
qdf['vote_count'] = qdf['vote_count'].astype('int')
qdf['vote_average'] = qdf['vote_average'].astype('int')
qdf.shape

(2459, 5)

In [14]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [15]:
qdf['wr'] = qdf.apply(weighted_rating, axis=1)

In [16]:
qdf = qdf.sort_values('wr', ascending=False)

In [17]:
qdf.head(50)


,title,Year,vote_count,vote_average,genres,wr
2418,The Matrix,1999.0,42120,4,"Action,Sci-Fi,Thriller",3.976584
308,The Shawshank Redemption,1994.0,40889,4,"Crime,Drama",3.975891
16141,Inception,2010.0,37947,4,"Action,Crime,Drama,Mystery,Sci-Fi,Thriller,IMAX",3.974059
13070,The Dark Knight,2008.0,34531,4,"Action,Crime,Drama,IMAX",3.971547
2801,Fight Club,1999.0,34290,4,"Action,Crime,Drama,Thriller",3.971351
345,Forrest Gump,1994.0,33854,4,"Comedy,Drama,Romance,War",3.970990
6927,The Lord of the Rings: The Return of the King,2003.0,32509,4,"Action,Adventure,Drama,Fantasy",3.969817
4804,The Lord of the Rings: The Fellowship of the Ring,2001.0,31634,4,"Adventure,Fantasy",3.969002
286,Pulp Fiction,1994.0,29140,4,"Comedy,Crime,Drama,Thriller",3.966416
5751,The Lord of the Rings: The Two Towers,2002.0,28750,4,"Adventure,Fantasy",3.965972


In [18]:
md['genres'] = md['genres'].fillna("")

In [19]:
md['genres1']=md.apply( lambda x : x['genres'].split(','), axis=1 )

In [20]:
md.head()

,movieId,unformatted_title,title,titlewithyear,releaseDate,Year,director,starring,imdbId,tmdbId,genres,summary,vote_average,vote_count,genres1
0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Adventure,Animation,Children,Comedy,Fantasy","Led by Woody, Andy's toys live happily in his ...",3.799308,20666,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji,Jumanji,Jumanji (1995),12/15/1995,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,"Adventure,Children,Fantasy",When siblings Judy and Peter discover an encha...,3.302115,6620,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men,Grumpier Old Men,Grumpier Old Men (1995),1/1/1995,1995.0,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,"Comedy,Romance",A family wedding reignites the ancient feud be...,2.943738,1022,"[Comedy, Romance]"
3,4,Waiting to Exhale,Waiting to Exhale,Waiting to Exhale (1995),1/15/1996,1996.0,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,31357.0,"Comedy,Drama,Romance","Cheated on, mistreated and stepped on, the wom...",2.736641,131,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II,Father of the Bride Part II,Father of the Bride Part II (1995),12/8/1995,1995.0,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,11862.0,Comedy,Just when George Banks has recovered from his ...,2.887511,2218,[Comedy]


### splitting genre into new lines and stacking

In [21]:
s = md.apply(lambda x: pd.Series(x['genres1']),axis=1).stack()


In [22]:
pd.Series(md.genres1[0])

0    Adventure
1    Animation
2     Children
3       Comedy
4      Fantasy
dtype: object

In [23]:
s.head(10)

0  0    Adventure
   1    Animation
   2     Children
   3       Comedy
   4      Fantasy
1  0    Adventure
   1     Children
   2      Fantasy
2  0       Comedy
   1      Romance
dtype: object

In [24]:
s = md.apply(lambda x: pd.Series(x['genres1']),axis=1).stack().reset_index(level=1, drop=True)


In [25]:
s

0          Adventure
0          Animation
0           Children
0             Comedy
0            Fantasy
1          Adventure
1           Children
1            Fantasy
2             Comedy
2            Romance
3             Comedy
3              Drama
3            Romance
4             Comedy
5             Action
5              Crime
5           Thriller
6             Comedy
6            Romance
7          Adventure
7           Children
8             Action
9             Action
9          Adventure
9           Thriller
10            Comedy
10             Drama
10           Romance
11            Comedy
11            Horror
            ...     
49130         Comedy
49131          Drama
49132       Thriller
49132         Horror
49132        Mystery
49133    Documentary
49134       Children
49135          Drama
49136          Drama
49137               
49138          Crime
49138       Thriller
49139        Romance
49139         Comedy
49140          Drama
49140         Comedy
49141        

In [26]:
s.name = 'genre'
expand = md.drop('genres', axis=1).join(s)

In [27]:
expand

,movieId,unformatted_title,title,titlewithyear,releaseDate,Year,director,starring,imdbId,tmdbId,summary,vote_average,vote_count,genres1,genre
0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Led by Woody, Andy's toys live happily in his ...",3.799308,20666,"[Adventure, Animation, Children, Comedy, Fantasy]",Adventure
0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Led by Woody, Andy's toys live happily in his ...",3.799308,20666,"[Adventure, Animation, Children, Comedy, Fantasy]",Animation
0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Led by Woody, Andy's toys live happily in his ...",3.799308,20666,"[Adventure, Animation, Children, Comedy, Fantasy]",Children
0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Led by Woody, Andy's toys live happily in his ...",3.799308,20666,"[Adventure, Animation, Children, Comedy, Fantasy]",Comedy
0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Led by Woody, Andy's toys live happily in his ...",3.799308,20666,"[Adventure, Animation, Children, Comedy, Fantasy]",Fantasy
1,2,Jumanji,Jumanji,Jumanji (1995),12/15/1995,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,When siblings Judy and Peter discover an encha...,3.302115,6620,"[Adventure, Children, Fantasy]",Adventure
1,2,Jumanji,Jumanji,Jumanji (1995),12/15/1995,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,When siblings Judy and Peter discover an encha...,3.302115,6620,"[Adventure, Children, Fantasy]",Children
1,2,Jumanji,Jumanji,Jumanji (1995),12/15/1995,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,When siblings Judy and Peter discover an encha...,3.302115,6620,"[Adventure, Children, Fantasy]",Fantasy
2,3,Grumpier Old Men,Grumpier Old Men,Grumpier Old Men (1995),1/1/1995,1995.0,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,A family wedding reignites the ancient feud be...,2.943738,1022,"[Comedy, Romance]",Comedy
2,3,Grumpier Old Men,Grumpier Old Men,Grumpier Old Men (1995),1/1/1995,1995.0,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,A family wedding reignites the ancient feud be...,2.943738,1022,"[Comedy, Romance]",Romance


# creating genre wise movie list 

In [28]:
def genre_list(genre, percentile=0.8):
    x=expand[expand['genre']==genre]
    
    vote_counts = x[x['vote_count'].notnull()]['vote_count'].astype('int')
    m = vote_counts.quantile(percentile)
    
    vote_averages = x[x['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    
    q = x[(x['vote_count'] >= m) & (x['vote_count'].notnull()) & (x['vote_average'].notnull())][['title', 'Year', 'vote_count', 'vote_average', 'genre']]
    q['vote_count'] = q['vote_count'].astype('int')
    q['vote_average'] = q['vote_average'].astype('int')
    
    q['wr'] = q.apply(lambda x1 : (x1['vote_count']/(x1['vote_count']+m) * x1['vote_average']) + (m/(m+x1['vote_count']) * C), axis=1)
    
    q = q.sort_values('wr', ascending=False)
    return q

In [29]:
genre_list('Action').head()

,title,Year,vote_count,vote_average,genre,wr
2418,The Matrix,1999.0,42120,4,Action,3.993783
16141,Inception,2010.0,37947,4,Action,3.993102
13070,The Dark Knight,2008.0,34531,4,Action,3.992423
2801,Fight Club,1999.0,34290,4,Action,3.992370
6927,The Lord of the Rings: The Return of the King,2003.0,32509,4,Action,3.991954


In [31]:
listofgenres=list(expand['genre'].value_counts().keys())

In [32]:
for x in listofgenres :
    print("                      ",x)
    print(genre_list(x)['title'].head(10))
    print("                                       ")

                       Drama
308                           The Shawshank Redemption
16141                                        Inception
13070                                  The Dark Knight
2801                                        Fight Club
345                                       Forrest Gump
6927     The Lord of the Rings: The Return of the King
286                                       Pulp Fiction
515                                   Schindler's List
807                                      The Godfather
2702                                   American Beauty
Name: title, dtype: object
                                       
                       Comedy
345                         Forrest Gump
286                         Pulp Fiction
4784                              Amelie
1121                  The Princess Bride
589                                Fargo
1067     Monty Python and the Holy Grail
3832                              Snatch
1611                    The Big Lebow

345                                           Forrest Gump
515                                       Schindler's List
25682                                   The Imitation Game
2172                                     Life Is Beautiful
5794                                           The Pianist
711      Dr. Strangelove or: How I Learned to Stop Worr...
1145                                     Full Metal Jacket
1131                                        Apocalypse Now
7243                                      Band of Brothers
5492                                Grave of the Fireflies
Name: title, dtype: object
                                       
                       Musical
353                       The Lion King
570                             Aladdin
7648                            Shrek 2
16278       Scott Pilgrim vs. the World
577                Beauty and the Beast
16807                           Tangled
6641                     School of Rock
22918                           

In [33]:
taggen.head()

,movieId,tag,relevance
0,1,007,0.02250
1,1,007 (series),0.02300
2,1,18th century,0.03650
3,1,1920s,0.07250
4,1,1930s,0.13725


### taking top 5 relevent tags for each movie

In [34]:
df1=taggen.sort_values(['movieId','relevance'],ascending=False).groupby('movieId').head(5)

In [35]:
df1

,movieId,tag,relevance
12413253,175197,original,0.84600
12412665,175197,books,0.81350
12412533,175197,adapted from:book,0.76625
12412618,175197,based on a book,0.75300
12412532,175197,adaptation,0.71700
12412035,174585,mindfuck,0.99850
12412244,174585,robots,0.99375
12411789,174585,franchise,0.98200
12411820,174585,giant robots,0.98200
12411402,174585,action,0.97650


In [36]:
df2=pd.DataFrame(df1.groupby('movieId')['tag'].apply(list))

In [37]:
df2

,tag
movieId,
1,"[toys, computer animation, kids and family, pi..."
2,"[adventure, jungle, kids, children, fantasy]"
3,"[good sequel, sequel, comedy, sequels, original]"
4,"[women, romantic, girlie movie, chick flick, u..."
5,"[father daughter relationship, good sequel, se..."
6,"[gunfight, crime, heist, bank robbery, great a..."
7,"[remake, romantic, romantic comedy, paris, rom..."
8,"[adventure, based on a book, adapted from:book..."
9,"[action, good action, lone hero, video game ad..."


In [38]:
x=pd.merge(md,df2,on='movieId')

In [39]:
x.head()

,movieId,unformatted_title,title,titlewithyear,releaseDate,Year,director,starring,imdbId,tmdbId,genres,summary,vote_average,vote_count,genres1,tag
0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Adventure,Animation,Children,Comedy,Fantasy","Led by Woody, Andy's toys live happily in his ...",3.799308,20666,"[Adventure, Animation, Children, Comedy, Fantasy]","[toys, computer animation, kids and family, pi..."
1,2,Jumanji,Jumanji,Jumanji (1995),12/15/1995,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,"Adventure,Children,Fantasy",When siblings Judy and Peter discover an encha...,3.302115,6620,"[Adventure, Children, Fantasy]","[adventure, jungle, kids, children, fantasy]"
2,3,Grumpier Old Men,Grumpier Old Men,Grumpier Old Men (1995),1/1/1995,1995.0,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,"Comedy,Romance",A family wedding reignites the ancient feud be...,2.943738,1022,"[Comedy, Romance]","[good sequel, sequel, comedy, sequels, original]"
3,4,Waiting to Exhale,Waiting to Exhale,Waiting to Exhale (1995),1/15/1996,1996.0,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,31357.0,"Comedy,Drama,Romance","Cheated on, mistreated and stepped on, the wom...",2.736641,131,"[Comedy, Drama, Romance]","[women, romantic, girlie movie, chick flick, u..."
4,5,Father of the Bride Part II,Father of the Bride Part II,Father of the Bride Part II (1995),12/8/1995,1995.0,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,11862.0,Comedy,Just when George Banks has recovered from his ...,2.887511,2218,[Comedy],"[father daughter relationship, good sequel, se..."


### director and cast 

In [40]:
x['director'] = x['director'].fillna("")
x['starring'] = x['starring'].fillna("")


x['director1']=x.apply( lambda x : x['director'].split(','), axis=1 )

x['starring1']=x.apply( lambda x : x['starring'].split(','), axis=1 )


In [41]:
x.head(20)

,movieId,unformatted_title,title,titlewithyear,releaseDate,Year,director,starring,imdbId,tmdbId,genres,summary,vote_average,vote_count,genres1,tag,director1,starring1
0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Adventure,Animation,Children,Comedy,Fantasy","Led by Woody, Andy's toys live happily in his ...",3.799308,20666,"[Adventure, Animation, Children, Comedy, Fantasy]","[toys, computer animation, kids and family, pi...",[John Lasseter],"[Tim Allen, Tom Hanks, Don Rickles, Jim Var..."
1,2,Jumanji,Jumanji,Jumanji (1995),12/15/1995,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,"Adventure,Children,Fantasy",When siblings Judy and Peter discover an encha...,3.302115,6620,"[Adventure, Children, Fantasy]","[adventure, jungle, kids, children, fantasy]",[Joe Johnston],"[Jonathan Hyde, Bradley Pierce, Robin Willia..."
2,3,Grumpier Old Men,Grumpier Old Men,Grumpier Old Men (1995),1/1/1995,1995.0,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,"Comedy,Romance",A family wedding reignites the ancient feud be...,2.943738,1022,"[Comedy, Romance]","[good sequel, sequel, comedy, sequels, original]",[Howard Deutch],"[Jack Lemmon, Walter Matthau, Ann-Margret , ..."
3,4,Waiting to Exhale,Waiting to Exhale,Waiting to Exhale (1995),1/15/1996,1996.0,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,31357.0,"Comedy,Drama,Romance","Cheated on, mistreated and stepped on, the wom...",2.736641,131,"[Comedy, Drama, Romance]","[women, romantic, girlie movie, chick flick, u...",[Forest Whitaker],"[Angela Bassett, Loretta Devine, Whitney Hou..."
4,5,Father of the Bride Part II,Father of the Bride Part II,Father of the Bride Part II (1995),12/8/1995,1995.0,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,11862.0,Comedy,Just when George Banks has recovered from his ...,2.887511,2218,[Comedy],"[father daughter relationship, good sequel, se...",[Charles Shyer],"[Steve Martin, Martin Short, Diane Keaton, ..."
5,6,Heat,Heat,Heat (1995),1/1/1995,1995.0,Michael Mann,"Robert De Niro, Al Pacino, Val Kilmer, Jon Voi...",113277,949.0,"Action,Crime,Thriller","Obsessive master thief, Neil McCauley leads a ...",3.973550,5085,"[Action, Crime, Thriller]","[gunfight, crime, heist, bank robbery, great a...",[Michael Mann],"[Robert De Niro, Al Pacino, Val Kilmer, Jon..."
6,7,Sabrina,Sabrina,Sabrina (1995),1/1/1995,1995.0,Sydney Pollack,"Harrison Ford, Greg Kinnear, Nancy Marchand, J...",114319,11860.0,"Comedy,Romance",An ugly duckling having undergone a remarkable...,3.272681,981,"[Comedy, Romance]","[remake, romantic, romantic comedy, paris, rom...",[Sydney Pollack],"[Harrison Ford, Greg Kinnear, Nancy Marchand..."
7,8,Tom and Huck,Tom and Huck,Tom and Huck (1995),12/22/1995,1995.0,Peter Hewitt,"Jonathan Taylor Thomas, Brad Renfro, Eric Schw...",112302,45325.0,"Adventure,Children","A mischievous young boy, Tom Sawyer, witnesses...",2.840336,119,"[Adventure, Children]","[adventure, based on a book, adapted from:book...",[Peter Hewitt],"[Jonathan Taylor Thomas, Brad Renfro, Eric S..."
8,9,Sudden Death,Sudden Death,Sudden Death (1995),12/22/1995,1995.0,Peter Hyams,"Raymond J. Barry, Powers Boothe, Jean-Claude V...",114576,9091.0,Action,International action superstar Jean Claude Van...,2.704797,271,[Action],"[action, good action, lone hero, video game ad...",[Peter Hyams],"[Raymond J. Barry, Powers Boothe, Jean-Claud..."
9,10,GoldenEye,GoldenEye,GoldenEye (1995),11/17/1995,1995.0,Martin Campbell,"Pierce Brosnan, Sean Bean, Famke Janssen, Izab...",113189,710.0,"Action,Adventure,Thriller",James Bond must unmask the mysterious head of ...,3.389957,4421,"[Action, Adventure, Thriller]","[007 (series), 007, bond, franchise, espionage]",[Martin Campbell],"[Pierce Brosnan, Sean Bean, Famke Janssen, ..."


###  creating keyword soup of director, cast, genres, tags

In [42]:
x['cgt']=x['director1']+x['director1']+x['starring1']+x['genres1']+x['tag']

In [43]:
x['cgt'] = x['cgt'].apply(lambda x: ' '.join(x))

### count vectorizer

In [44]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(x['cgt'])

### cosine similarity

In [45]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [46]:
cosine_sim

array([[1.        , 0.11044205, 0.02871537, ..., 0.01227849, 0.06542239,
        0.02573638],
       [0.11044205, 1.        , 0.        , ..., 0.        , 0.03576564,
        0.03517444],
       [0.02871537, 0.        , 1.        , ..., 0.        , 0.        ,
        0.01829098],
       ...,
       [0.01227849, 0.        , 0.        , ..., 1.        , 0.07952553,
        0.        ],
       [0.06542239, 0.03576564, 0.        , ..., 0.07952553, 1.        ,
        0.05000695],
       [0.02573638, 0.03517444, 0.01829098, ..., 0.        , 0.05000695,
        1.        ]])

In [47]:
x = x.reset_index()
titles = x['title']
indices = pd.Series(x.index, index=x['title'])

In [48]:
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [49]:
indices['Pulp Fiction']

278

In [50]:
list(enumerate(cosine_sim[278]))

[(0, 0.051899929610768204),
 (1, 0.0),
 (2, 0.024590370452110582),
 (3, 0.04820676840414581),
 (4, 0.033058980245364314),
 (5, 0.03154401489382559),
 (6, 0.03615507630310936),
 (7, 0.03359684204526465),
 (8, 0.02203932016357621),
 (9, 0.009562921842487448),
 (10, 0.03361463227264072),
 (11, 0.03849001794597505),
 (12, 0.0),
 (13, 0.028947228095494784),
 (14, 0.012051692101036453),
 (15, 0.0627702980962258),
 (16, 0.023644230071063622),
 (17, 0.28488492464476783),
 (18, 0.049180740904221164),
 (19, 0.05824240056825374),
 (20, 0.08963901939370858),
 (21, 0.045599037870451296),
 (22, 0.07377111135633174),
 (23, 0.012051692101036453),
 (24, 0.012830005981991684),
 (25, 0.012295185226055291),
 (26, 0.017299976536922734),
 (27, 0.024103384202072906),
 (28, 0.018044398732025558),
 (29, 0.03615507630310936),
 (30, 0.03569863684011106),
 (31, 0.07064172571013567),
 (32, 0.0),
 (33, 0.011204877424213574),
 (34, 0.05021623847698063),
 (35, 0.07051308660847494),
 (36, 0.024590370452110582),
 (37, 

# content based recommender system 1 : using director, cast, tags and genres

In [51]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [52]:
get_recommendations('The Hateful Eight').head(10)

963           Reservoir Dogs
6182       Kill Bill: Vol. 2
9731        Django Unchained
8686    Inglourious Basterds
8097             Death Proof
278             Pulp Fiction
5779       Kill Bill: Vol. 1
1496            Jackie Brown
8037              Grindhouse
17                Four Rooms
Name: title, dtype: object

In [53]:
x.head()

,index,movieId,unformatted_title,title,titlewithyear,releaseDate,Year,director,starring,imdbId,tmdbId,genres,summary,vote_average,vote_count,genres1,tag,director1,starring1,cgt
0,0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Adventure,Animation,Children,Comedy,Fantasy","Led by Woody, Andy's toys live happily in his ...",3.799308,20666,"[Adventure, Animation, Children, Comedy, Fantasy]","[toys, computer animation, kids and family, pi...",[John Lasseter],"[Tim Allen, Tom Hanks, Don Rickles, Jim Var...",John Lasseter John Lasseter Tim Allen Tom Han...
1,1,2,Jumanji,Jumanji,Jumanji (1995),12/15/1995,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,"Adventure,Children,Fantasy",When siblings Judy and Peter discover an encha...,3.302115,6620,"[Adventure, Children, Fantasy]","[adventure, jungle, kids, children, fantasy]",[Joe Johnston],"[Jonathan Hyde, Bradley Pierce, Robin Willia...",Joe Johnston Joe Johnston Jonathan Hyde Bradl...
2,2,3,Grumpier Old Men,Grumpier Old Men,Grumpier Old Men (1995),1/1/1995,1995.0,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,"Comedy,Romance",A family wedding reignites the ancient feud be...,2.943738,1022,"[Comedy, Romance]","[good sequel, sequel, comedy, sequels, original]",[Howard Deutch],"[Jack Lemmon, Walter Matthau, Ann-Margret , ...",Howard Deutch Howard Deutch Jack Lemmon Walte...
3,3,4,Waiting to Exhale,Waiting to Exhale,Waiting to Exhale (1995),1/15/1996,1996.0,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,31357.0,"Comedy,Drama,Romance","Cheated on, mistreated and stepped on, the wom...",2.736641,131,"[Comedy, Drama, Romance]","[women, romantic, girlie movie, chick flick, u...",[Forest Whitaker],"[Angela Bassett, Loretta Devine, Whitney Hou...",Forest Whitaker Forest Whitaker Angela Bassett...
4,4,5,Father of the Bride Part II,Father of the Bride Part II,Father of the Bride Part II (1995),12/8/1995,1995.0,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,11862.0,Comedy,Just when George Banks has recovered from his ...,2.887511,2218,[Comedy],"[father daughter relationship, good sequel, se...",[Charles Shyer],"[Steve Martin, Martin Short, Diane Keaton, ...",Charles Shyer Charles Shyer Steve Martin Mart...


### using summary

In [54]:
x['summary'] = x['summary'].fillna("")

In [55]:
x.summary[28]

''

In [56]:
x['summary1']=x.apply( lambda x : x['summary'].split(' '), axis=1 )

### stemming words

In [57]:
stemmer = SnowballStemmer('english')
stemmer.stem('toys')

'toy'

In [58]:
stemmer.stem('happily')

'happili'

In [59]:
x['keywords'] = x['summary1'].apply(lambda x: [stemmer.stem(i) for i in x])


In [60]:
x['keywords1'] = x['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [61]:
x['summary_new']=x['keywords1'].apply(' '.join)

### tfidf vectorizer to remove common words (like stop words)

In [62]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(x['summary_new'])

In [63]:
cosine_sim_tf = cosine_similarity(tfidf_matrix, tfidf_matrix)

# content based recommender system 2 : using keywords

In [64]:
def get_recommendations_tfidf(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim_tf[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [65]:
get_recommendations('The Hateful Eight').head(10)

963           Reservoir Dogs
6182       Kill Bill: Vol. 2
9731        Django Unchained
8686    Inglourious Basterds
8097             Death Proof
278             Pulp Fiction
5779       Kill Bill: Vol. 1
1496            Jackie Brown
8037              Grindhouse
17                Four Rooms
Name: title, dtype: object

In [66]:
get_recommendations_tfidf('The Hateful Eight').head(10)

9513                                     One for the Money
6614                                          The Tin Star
7505                                                Domino
9731                                      Django Unchained
4190                                            The Hunter
9529                                            The Hunter
4085                                         Pink Cadillac
10916    The Rocky Horror Picture Show: Let's Do the Ti...
3335                                              Freejack
1222                                                  Jaws
Name: title, dtype: object

In [67]:
list(enumerate(cosine_sim[0]))

[(0, 0.9999999999999988),
 (1, 0.11044204887501098),
 (2, 0.0287153661502632),
 (3, 0.014073344364024968),
 (4, 0.025736381216072766),
 (5, 0.012278491144729288),
 (6, 0.028146688728049936),
 (7, 0.029424494316824992),
 (8, 0.0),
 (9, 0.022334173672426936),
 (10, 0.026168955732628656),
 (11, 0.02996443833169965),
 (12, 0.2111001654603745),
 (13, 0.12169108714722551),
 (14, 0.028146688728049936),
 (15, 0.0),
 (16, 0.027610512218752745),
 (17, 0.023762422826087877),
 (18, 0.0430730492253948),
 (19, 0.009716067288818148),
 (20, 0.06542238933157164),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.010101010101010102),
 (27, 0.0),
 (28, 0.0316069770620507),
 (29, 0.0),
 (30, 0.12506107989614448),
 (31, 0.0),
 (32, 0.027610512218752745),
 (33, 0.026168955732628656),
 (34, 0.0),
 (35, 0.05881538943671546),
 (36, 0.0574307323005264),
 (37, 0.05233791146525731),
 (38, 0.01246595986677943),
 (39, 0.0),
 (40, 0.012278491144729288),
 (41, 0.01704923897625215),
 (42, 0.0),
 (43,

In [68]:
list(enumerate(cosine_sim_tf[0]))

[(0, 0.9999999999999978),
 (1, 0.01011831337066707),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.01915539881522548),
 (18, 0.0),
 (19, 0.0),
 (20, 0.003471218835077425),
 (21, 0.0),
 (22, 0.0),
 (23, 0.002955295607356213),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.002538577554680049),
 (31, 0.0),
 (32, 0.0),
 (33, 0.00811364325797467),
 (34, 0.0),
 (35, 0.0),
 (36, 0.003836106131392911),
 (37, 0.005889147500116653),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.0016003982182899506),
 (43, 0.0),
 (44, 0.0),
 (45, 0.0032954264579930894),
 (46, 0.009218612928534967),
 (47, 0.0),
 (48, 0.0),
 (49, 0.007091400715026264),
 (50, 0.008784263589034884),
 (51, 0.0),
 (52, 0.0),
 (53, 0.00907870820058602),
 (54, 0.0),
 (55, 0.007118293990259588),
 (56, 0.0),
 (57, 0.0),
 (58, 0.015986250780072354),
 (59, 0.0),
 (60,

In [69]:
cosine=(cosine_sim+cosine_sim_tf)/2

In [70]:
list(enumerate(cosine[0]))

[(0, 0.9999999999999982),
 (1, 0.060280181122839026),
 (2, 0.0143576830751316),
 (3, 0.007036672182012484),
 (4, 0.012868190608036383),
 (5, 0.006139245572364644),
 (6, 0.014073344364024968),
 (7, 0.014712247158412496),
 (8, 0.0),
 (9, 0.011167086836213468),
 (10, 0.013084477866314328),
 (11, 0.014982219165849825),
 (12, 0.10555008273018725),
 (13, 0.060845543573612756),
 (14, 0.014073344364024968),
 (15, 0.0),
 (16, 0.013805256109376373),
 (17, 0.02145891082065668),
 (18, 0.0215365246126974),
 (19, 0.004858033644409074),
 (20, 0.03444680408332453),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0014776478036781066),
 (24, 0.0),
 (25, 0.0),
 (26, 0.005050505050505051),
 (27, 0.0),
 (28, 0.01580348853102535),
 (29, 0.0),
 (30, 0.06379982872541226),
 (31, 0.0),
 (32, 0.013805256109376373),
 (33, 0.017141299495301663),
 (34, 0.0),
 (35, 0.02940769471835773),
 (36, 0.030633419215959655),
 (37, 0.029113529482686983),
 (38, 0.006232979933389715),
 (39, 0.0),
 (40, 0.006139245572364644),
 (41, 0.00852461948

# content based recommender system 3 : hybrid of recommender 1 and 2

In [71]:
def get_reco(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [72]:
super=pd.merge(x,md,on='movieId')

In [73]:
super.head()

,index,movieId,unformatted_title_x,title_x,titlewithyear_x,releaseDate_x,Year_x,director_x,starring_x,imdbId_x,...,Year_y,director_y,starring_y,imdbId_y,tmdbId_y,genres_y,summary_y,vote_average_y,vote_count_y,genres1_y
0,0,1,Toy Story,Toy Story,Toy Story (1995),11/19/1995,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,...,1995.0,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Adventure,Animation,Children,Comedy,Fantasy","Led by Woody, Andy's toys live happily in his ...",3.799308,20666,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1,2,Jumanji,Jumanji,Jumanji (1995),12/15/1995,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,...,1995.0,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,"Adventure,Children,Fantasy",When siblings Judy and Peter discover an encha...,3.302115,6620,"[Adventure, Children, Fantasy]"
2,2,3,Grumpier Old Men,Grumpier Old Men,Grumpier Old Men (1995),1/1/1995,1995.0,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,...,1995.0,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,"Comedy,Romance",A family wedding reignites the ancient feud be...,2.943738,1022,"[Comedy, Romance]"
3,3,4,Waiting to Exhale,Waiting to Exhale,Waiting to Exhale (1995),1/15/1996,1996.0,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,...,1996.0,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,31357.0,"Comedy,Drama,Romance","Cheated on, mistreated and stepped on, the wom...",2.736641,131,"[Comedy, Drama, Romance]"
4,4,5,Father of the Bride Part II,Father of the Bride Part II,Father of the Bride Part II (1995),12/8/1995,1995.0,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,...,1995.0,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,11862.0,Comedy,Just when George Banks has recovered from his ...,2.887511,2218,[Comedy]


# content based recommender system 4 : recommender 3 sorted by popularity

In [74]:
def get_best_reco(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    
    mo = super.iloc[movie_indices][['title_x', 'vote_count_y', 'vote_average_y']]
    
    vote_counts = super[super['vote_count_y'].notnull()]['vote_count_y'].astype('int')
    m = vote_counts.quantile(0.5)
    
    vote_averages = super[super['vote_average_y'].notnull()]['vote_average_y'].astype('int')
    C = vote_averages.mean()
   
    q = mo[(mo['vote_count_y'] >= m) & (mo['vote_count_y'].notnull()) & (mo['vote_average_y'].notnull())][['title_x', 'vote_count_y','vote_average_y']]
    
    q['vote_count_y'] = q['vote_count_y'].astype('int')
    q['vote_average_y'] = q['vote_average_y'].astype('int')
    
    q['wr'] = q.apply(lambda x1 : (x1['vote_count_y']/(x1['vote_count_y']+m) * x1['vote_average_y']) + (m/(m+x1['vote_count_y']) * C), axis=1)
    
    q = q.sort_values('wr', ascending=False)
    
   
    return q['title_x']

In [75]:
def four_reco(title, num):
    
        ans=pd.concat([pd.DataFrame(get_best_reco(title).head(num).reset_index(drop=True)),
                       pd.DataFrame(get_reco(title).head(num).reset_index(drop=True)),
                       pd.DataFrame(get_recommendations(title).head(num).reset_index(drop=True)),
                   pd.DataFrame(get_recommendations_tfidf(title).head(num).reset_index(drop=True))
                  
                               ]
                  , axis=1)
        ans.columns=['4. BEST_RECOMMENDATION-Cast_Tags_Summary_Popularity', '3. Cast_Tags_Summary','1. Cast_Tags', '2. Summary' ]
        
        return ans

In [76]:
four_reco('The Hateful Eight',15)

,4. BEST_RECOMMENDATION-Cast_Tags_Summary_Popularity,3. Cast_Tags_Summary,1. Cast_Tags,2. Summary
0,Pulp Fiction,Django Unchained,Reservoir Dogs,One for the Money
1,Django Unchained,Reservoir Dogs,Kill Bill: Vol. 2,The Tin Star
2,Reservoir Dogs,Kill Bill: Vol. 2,Django Unchained,Domino
3,Inglourious Basterds,Kill Bill: Vol. 1,Inglourious Basterds,Django Unchained
4,Kill Bill: Vol. 1,Inglourious Basterds,Death Proof,The Hunter
5,Kill Bill: Vol. 2,Death Proof,Pulp Fiction,The Hunter
6,Jackie Brown,Pulp Fiction,Kill Bill: Vol. 1,Pink Cadillac
7,Grindhouse,Jackie Brown,Jackie Brown,The Rocky Horror Picture Show: Let's Do the Ti...
8,Death Proof,Grindhouse,Grindhouse,Freejack
9,Four Rooms,Four Rooms,Four Rooms,Jaws


In [77]:
four_reco('The Dark Knight',10)

,4. BEST_RECOMMENDATION-Cast_Tags_Summary_Popularity,3. Cast_Tags_Summary,1. Cast_Tags,2. Summary
0,Inception,The Dark Knight Rises,Batman Begins,The Dark Knight Rises
1,Interstellar,Batman Begins,The Dark Knight Rises,Batman Forever
2,The Prestige,Batman Returns,Batman Returns,Batman Returns
3,Batman Begins,Batman,Inception,Batman
4,The Dark Knight Rises,Batman Forever,Dunkirk,Batman
5,Million Dollar Baby,Dunkirk,The Prestige,Batman: Under the Red Hood
6,Batman,Batman & Mr. Freeze: Subzero,Batman,"Batman: The Dark Knight Returns, Part 2"
7,The Deer Hunter,Inception,Insomnia,JFK
8,Batman Returns,The Prestige,The Deer Hunter,Batman: Year One
9,Insomnia,Insomnia,Doodlebug,Q & A


In [78]:
four_reco('Jumanji',10)

,4. BEST_RECOMMENDATION-Cast_Tags_Summary_Popularity,3. Cast_Tags_Summary,1. Cast_Tags,2. Summary
0,Captain America: The First Avenger,"Honey, I Shrunk the Kids","Honey, I Shrunk the Kids",Word Wars
1,The NeverEnding Story,Hidalgo,Hidalgo,Doctor Who: Last Christmas
2,The Rocketeer,The Rocketeer,The Rocketeer,Indie Game: The Movie
3,Bridge to Terabithia,Peter Pan,Pan,Guardians of the Galaxy
4,Hidalgo,Pan,Return to Oz,Brainscan
5,Peter Pan,Alice Through the Looking Glass,The Wolfman,Wreck-It Ralph
6,Alice Through the Looking Glass,The NeverEnding Story II: The Next Chapter,Alice Through the Looking Glass,The Last of Sheila
7,Escape to Witch Mountain,Return to Oz,The NeverEnding Story II: The Next Chapter,eXistenZ
8,Return to Oz,The Wolfman,The NeverEnding Story,Stay Alive
9,The NeverEnding Story III,The NeverEnding Story,Escape to Witch Mountain,Pixels


# collaborative filtering : recommender system based on user tastes

In [79]:
reader = Reader()
data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

### evaluating SVD

In [82]:
svd = SVD()

In [83]:
# svd = SVD()
# evaluate(svd, data, measures=['RMSE', 'MAE'])

# Evaluating RMSE, MAE of algorithm SVD.

# ------------
# Fold 1
# RMSE: 0.7800
# MAE:  0.5774
# ------------
# Fold 2
# RMSE: 0.7796
# MAE:  0.5775
# ------------
# Fold 3
# RMSE: 0.7795
# MAE:  0.5773
# ------------
# Fold 4
# RMSE: 0.7800
# MAE:  0.5778
# ------------
# Fold 5
# RMSE: 0.7792
# MAE:  0.5773
# ------------
# ------------
# Mean RMSE: 0.7797
# Mean MAE : 0.5775
# ------------
# ------------
# CaseInsensitiveDefaultDict(list,
#                            {'rmse': [0.7799855218913966,
#                              0.7796097861877095,
#                              0.7794901476579066,
#                              0.7800198793509012,
#                              0.7792220813642221],
#                             'mae': [0.5774450627472922,
#                              0.5774922484933642,
#                              0.5773451155037247,
#                              0.5777897382837646,
#                              0.5772634955929802]})

### training SVD

In [84]:
trainset = data.build_full_trainset()
svd.train(trainset)

# creating hybrid recommender system: using content based recommender system with director, cast, genres, tags, summary and also tailored to induvidual users' tastes

In [85]:
def hybrid(userId, title):
    idx = indices[title]
    
    sim_scores = list(enumerate(cosine[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    mo = super.iloc[movie_indices][['movieId','title_x']]
    t=train[train['userId']==userId]
    df = pd.merge(mo,t,on='movieId', how='left')
    #df['est'] = df['movieId'].apply(lambda x: svd.predict(userId, df.loc[x]['movieId']).est)
    df['est']= 0.00000
    
    for i in range(0,df.shape[0]):
        df.est[i]=svd.predict(df.userId[i], df.movieId[i]).est
    
    df=df.sort_values('est', ascending=False)
    
    return df['title_x']

In [86]:
hybrid(142882, 'Jumanji')

24                                     The Gnome-Mobile
14                                 Bridge to Terabithia
19                   Darby O'Gill and the Little People
12                   Captain America: The First Avenger
13                                            Tall Tale
7                                          Return to Oz
1                                               Hidalgo
11                             Escape to Witch Mountain
9                                 The NeverEnding Story
16             The New Adventures of Pippi Longstocking
20                            The Spiderwick Chronicles
21    The Chronicles of Narnia: The Voyage of the Da...
2                                         The Rocketeer
5                       Alice Through the Looking Glass
22             The Chronicles of Narnia: Prince Caspian
6            The NeverEnding Story II: The Next Chapter
15                                   Warriors of Virtue
8                                           The 

In [87]:
hybrid(142882, 'The Hateful Eight').head(10)

11            The Tin Star
6             Pulp Fiction
22          The Naked Spur
9               Four Rooms
21    Angel and the Badman
5              Death Proof
19           Flesh & Blood
12               The Krays
8               Grindhouse
1           Reservoir Dogs
Name: title_x, dtype: object

In [88]:
hybrid(142882, 'The Usual Suspects').head(10)

7                The Game
19             Paper Moon
0              No Way Out
21               Identity
24          Suicide Kings
18              Apt Pupil
10            Black Widow
16    Along Came a Spider
13          Secret Window
2                 13 Sins
Name: title_x, dtype: object

In [89]:
hybrid(142884, 'Mystic River').head(10)

1        Million Dollar Baby
0                Gran Torino
3                 Unforgiven
20       High Plains Drifter
18    The Outlaw Josey Wales
2            A Perfect World
24     Letters from Iwo Jima
9                 Pale Rider
4                   Invictus
8             Absolute Power
Name: title_x, dtype: object

# Thank you